In [1]:
from unityagents import UnityEnvironment
import numpy as np
import random

from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent
from model import Actor, Critic

import torch


The device to use:  cuda:0


In [2]:
env = UnityEnvironment(file_name='./Reacher_Linux_20/Reacher.x86_64')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.

In [3]:
scores = np.zeros(num_agents)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

random_seed=0

agent = Agent(state_size, action_size, random_seed=0)
agent.actor_local = Actor(state_size, action_size, random_seed).to(device)

agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))


In [4]:

while True:
    actions = agent.act(states)

    env_info = env.step(actions)[brain_name]        # send the action to the environment
    next_states = env_info.vector_observations     # get the next state
    rewards = env_info.rewards                     # get the reward
    dones = env_info.local_done        

    states = next_states
    
    scores += np.array(env_info.rewards) 

    print('\rScore: {:.2f}\tmin: {:.2f}\tmax: {:.2f}'
              .format(np.mean(scores), np.min(scores), np.max(scores)), end="") 
    
    if np.any(dones):
        break
        
print("\nScores: {}".format(scores))




Score: 37.86	min: 35.66	max: 38.96
Scores: [38.03999915 37.39999916 37.72999916 38.67999914 37.44999916 38.95999913
 37.48999916 38.14999915 38.87999913 36.76999918 37.37999916 38.78999913
 37.92999915 37.73999916 37.10999917 38.42999914 38.27999914 37.88999915
 35.6599992  38.34999914]


In [5]:
env.reset()
env.close()